## 1.0 Bibliotecas

In [1]:
import unidecode
import pandas as pd
import seaborn as sns
from os import listdir
from os.path import isfile, join
from IPython.display import HTML
from sklearn.preprocessing import minmax_scale

### 2.0 Funções

In [2]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    plt.style.use('bmh')
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set()
    
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


### 2.0 Constantes

In [3]:
cols = ['Total', 'Pelo menos uma',
                  'Visual-Não consegue de modo algum', 'Visual-Grande  dificuldade', 'Visual-Alguma dificuldade',
                  'Auditiva-Não consegue de modo algum', 'Auditiva-Grande  dificuldade', 'Auditiva-Alguma dificuldade',
                  'Motora-Não consegue de modo algum', 'Motora-Grande  dificuldade', 'Motora-Alguma dificuldade',
                  'Mental', 'Nenhuma', 'Estado']

cols_order = ['Estado', 'Total', 'Pelo menos uma',
                  'Visual-Não consegue de modo algum', 'Visual-Grande  dificuldade', 'Visual-Alguma dificuldade',
                  'Auditiva-Não consegue de modo algum', 'Auditiva-Grande  dificuldade', 'Auditiva-Alguma dificuldade',
                  'Motora-Não consegue de modo algum', 'Motora-Grande  dificuldade', 'Motora-Alguma dificuldade',
                  'Mental', 'Nenhuma']

### 3.0 Leitura de Dados

#### 3.1 Dados do Censo IBGE 2010 (Pessoas com Deficiência)
 - Fonte: https://www.ibge.gov.br/estatisticas/sociais/populacao/2098-np-censo-demografico/9662-censo-demografico-2010.html?edicao=9749&t=resultados

In [4]:
df_final = pd.DataFrame(columns=cols_order)

for estado in listdir('data'):
    url = 'data\\{}\\Tabela {}.3.1.xls'
    
    file_prefix = listdir('data\\{}'.format(estado))[0][7:11]
    file_prefix = file_prefix if file_prefix[-1] != '.' else file_prefix[:-1]
    
    url = url.format(estado,file_prefix)
    
    df = pd.read_excel(url, skiprows=7, header=None)
    df.drop(columns=[0,9], inplace=True)
    df['Estado'] = estado
    
    df.columns = cols
    df = df[cols_order]
    
    df_final = df_final.append(df.iloc[0])

#### 3.2 Dados do Censo IBGE 2010 (população por município)
 - Fonte: https://www.ibge.gov.br/estatisticas/sociais/saude/9662-censo-demografico-2010.html?=&t=resultados

In [5]:
df_pop_mun = pd.read_excel('pop_mun_ibge_2010.xlsx')

#### 3.3 Dados IBP Fiocruz 
- Fonte IBP: site da Fiocruz (https://cidacs.bahia.fiocruz.br/ibp/#) > Painel > IBP Municipal (download da base) > Formulário para download (https://docs.google.com/forms/d/e/1FAIpQLSdyzl4lXeuYnQJshhTyZ5_2Ejdd1xl_urfsOU2N2y6ax2oGUw/viewform)

In [6]:
df_ibp = pd.read_csv( 'IBP/BDI_Municipalities-Level_Short.csv' ).drop(columns=['uf', 'nome_da_uf', 'codmun6', 'nome_do_municipio'] )

#### 3.4 Dados dos municípios do Brasil com Latitude e Longitude
- Fonte da dados: https://github.com/kelvins/Municipios-Brasileiros/blob/main/csv/municipios.csv

In [7]:
df_mun = pd.read_csv( 'municipios.csv' ).drop( columns=['nome', 'capital', 'codigo_uf', 'siafi_id', 'ddd', 'fuso_horario'] )
df_mun.columns = ['c_cod_mun', 'lat', 'long']

## 4.0 Conversão de Tipo

In [8]:
for c in ['Total', 'Pelo menos uma', 'Visual-Não consegue de modo algum',
       'Visual-Grande  dificuldade', 'Visual-Alguma dificuldade',
       'Auditiva-Não consegue de modo algum', 'Auditiva-Grande  dificuldade',
       'Auditiva-Alguma dificuldade', 'Motora-Não consegue de modo algum',
       'Motora-Grande  dificuldade', 'Motora-Alguma dificuldade', 'Mental',
       'Nenhuma']:
    df_final[c] = df_final[c].astype('int')

### 5.0 Engenharia de Features

#### 5.1 Total de pessoas c/ repetição entre pessoas com mais de uma deficiência

In [9]:
df_final['Total c/ Repetição'] = df_final.iloc[:, 3:].apply(lambda x: x.sum(), axis=1)

#### 5.2 Total de pessoas c/ deficiência c/ repetição entre pessoas com mais de uma deficiência

In [10]:
df_final['Total c/ Deficiência c/ Repetição'] = df_final.iloc[:, 3:-2].apply(lambda x: x.sum(), axis=1)

#### 5.3 Percentual de pessoas com deficiência

In [11]:
df_final['% respostas com deficiência'] = round(df_final['Total c/ Deficiência c/ Repetição']/df_final['Total c/ Repetição'],4)*100

In [12]:
df_final['% pessoas com deficiência'] = round(df_final['Pelo menos uma']/df_final['Total'],4)*100

### 5.4 Coluna UF

In [13]:
d_uf = {
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AP': 'Amapá',
    'AM': 'Amazonas',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MT': 'Mato Grosso',
    'MS': 'Mato Grosso do Sul',
    'MG': 'Minas Gerais',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PR': 'Paraná',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'SC': 'Santa Catarina',
    'SP': 'São Paulo',
    'SE': 'Sergipe',
    'TO': 'Tocantins'
}
d_uf = {unidecode.unidecode(v): k for k, v in d_uf.items()}

In [14]:
df_final['UF'] = df_final['Estado'].map(d_uf)
df_final = df_final[['UF'] + list(df_final.columns[:-1])]

#### 5.5 Expansão da Granularidade da Base (Estado p/ Município)

In [15]:
df_pop_mun = df_pop_mun[['uf', 'cod_mun', 'mun', 'pop_abs', 'pop_rel_uf']]
df_pop_mun.columns = ['UF', 'c_cod_mun', 'c_mun', 'c_pop_abs', 'c_pop_rel_uf']
df_municipios = df_pop_mun.merge(df_final, how='left', on='UF')

In [16]:
df_municipios.head()

,UF,c_cod_mun,c_mun,c_pop_abs,c_pop_rel_uf,Estado,Total,Pelo menos uma,Visual-Não consegue de modo algum,Visual-Grande dificuldade,Visual-Alguma dificuldade,Auditiva-Não consegue de modo algum,Auditiva-Grande dificuldade,Auditiva-Alguma dificuldade,Motora-Não consegue de modo algum,Motora-Grande dificuldade,Motora-Alguma dificuldade,Mental,Nenhuma,Total c/ Repetição,Total c/ Deficiência c/ Repetição,% respostas com deficiência,% pessoas com deficiência
0,AC,1200013,Acrelândia,12538,0.017092,Acre,733558,165892,1410,25143,106849,1188,6159,26811,2105,11722,31560,12105,567462,792514,225052,28.4,22.61
1,AC,1200054,Assis Brasil,6072,0.008277,Acre,733558,165892,1410,25143,106849,1188,6159,26811,2105,11722,31560,12105,567462,792514,225052,28.4,22.61
2,AC,1200104,Brasiléia,21398,0.029170,Acre,733558,165892,1410,25143,106849,1188,6159,26811,2105,11722,31560,12105,567462,792514,225052,28.4,22.61
3,AC,1200138,Bujari,8471,0.011548,Acre,733558,165892,1410,25143,106849,1188,6159,26811,2105,11722,31560,12105,567462,792514,225052,28.4,22.61
4,AC,1200179,Capixaba,8798,0.011994,Acre,733558,165892,1410,25143,106849,1188,6159,26811,2105,11722,31560,12105,567462,792514,225052,28.4,22.61


#### 5.6 Métricas (Nessa ordem)
 - Quantidade de pessoas com deficiência por município
 - % marcações de deficiência (respostas) sobre o total de respostas recebidas
 - % população c/ deficiência por população da UF
 - % população c/ deficiência em relação a população total do Brasil
 - taxa da população c/ deficiência por população da UF
 - taxa da população c/ deficiência em relação a população total do Brasil
 - taxa da população c/ deficiência por população da UF (Normalizada)
 - taxa da população c/ deficiência em relação a população total do Brasil (Normalizada)

In [17]:
pop_bras_2010 = df_final['Total'].sum()

df_municipios['qtd. pessoas c/ deficiencia'] = round(df_municipios['c_pop_rel_uf'] * df_municipios['Pelo menos uma'])
df_municipios['% respostas com deficiência'] = round((df_municipios['c_pop_rel_uf'] * df_municipios['Pelo menos uma'])/df_municipios['c_pop_abs'],4)*100
df_municipios['c_%_pop_def_rel_uf'] = df_municipios['c_pop_rel_uf'] * df_municipios['Pelo menos uma'] / df_municipios['Total'] *100
df_municipios['c_%_pop_def_rel_bras'] = df_municipios['c_pop_rel_uf'] * df_municipios['Pelo menos uma'] / pop_bras_2010 *100

df_municipios['taxa_pop_def_reluf'] = df_municipios['c_%_pop_def_rel_uf']/100
df_municipios['taxa_pop_def_relbras'] = df_municipios['c_%_pop_def_rel_bras']/100


df_municipios['taxa_pop_def_reluf_norm'] = minmax_scale(df_municipios['taxa_pop_def_reluf'])
df_municipios['taxa_pop_def_relbras_norm'] = minmax_scale(df_municipios['taxa_pop_def_relbras'])

### 5.6 Concatenação de Dados

#### 5.6.1 Dados do IBP e IBP Normalizado
 - Valor do IBP normalizado.

In [18]:
df_municipios = df_municipios.merge(df_ibp, how='left', left_on='c_cod_mun', right_on='cod_municipio' )
df_municipios = df_municipios.rename( columns={'measure_1f_12': 'ibp', 'd_measure_1f_12': 'd_ibp', 'q_measure_1f_12': 'q_ibp' } )
df_municipios.drop( columns=['cod_municipio'], inplace=True )

df_municipios['ibp_norm'] = minmax_scale(df_municipios['ibp'])

#### 5.6.2 Latitude e Longitude dos Municípios

In [19]:
df_municipios = df_municipios.merge(df_mun, how='left', on='c_cod_mun' )

In [20]:
df_municipios['lat_long'] = df_mun['lat'].astype(str) + ',' + df_mun['long'].astype(str)

## 6.0 Salvar Dados em Arquivo CSV

In [22]:
df_municipios.to_csv( 'df_all_ibp.csv', decimal=',', index=False)

In [ ]:
# Abaixo, o parâmetro setado em 0.4 é o parâmetro 1 no dashboard. O segundo (0.6) é o parâmetro 2.
# Por construção, exige-se: 'parâmetro 1 + parâmetro 2 = 1'
df_municipios['taxa'] = df_municipios['taxa_pop_def_relbras_norm']*0.4 + df_municipios['ibp_norm']*0.6

In [ ]:
df_municipios['taxa'].quantile(0.75)

In [ ]:
pd.Series(minmax_scale(df_municipios['taxa'])).max()